## Preliminary
Get our tools

In [1]:
from iexfinance.stocks import Stock, get_historical_data

import pandas as pd
import numpy as np

from datetime import datetime

## Does Bigger Company have Less Risk?

## Let's import out Nasdaq 100 constituents from [here](https://www.nasdaq.com/quotes/nasdaq-100-stocks.aspx)

In [2]:
fname = "data/nasdaq100list.csv"
nasdaq_sym = pd.read_csv(fname)
nasdaq_sym.head()

,Symbol,Name,lastsale,netchange,pctchange,share_volume,Nasdaq100_points,Unnamed: 7
0,ATVI,Activision Blizzard Inc,47.29,0.33,0.70,5976678,0.2,NaN
1,ADBE,Adobe Inc.,267.45,-0.44,-0.16,1970627,-0.2,NaN
2,AMD,Advanced Micro Devices Inc.,28.98,-0.11,-0.38,65662738,-0.1,NaN
3,ALXN,Alexion Pharmaceuticals Inc.,141.20,1.27,0.91,1208313,0.3,NaN
4,ALGN,Align Technology Inc.,295.88,7.92,2.75,1386161,0.6,NaN


In [3]:
nasdaq_sym.columns = [colname.strip() for colname in list(nasdaq_sym.columns)]
nasdaq_sym = nasdaq_sym.drop(columns = ['Unnamed: 7'])
nasdaq_sym['cap'] = nasdaq_sym.apply(lambda x : x['share_volume'] * x['lastsale'],axis =1)
nasdaq_sym['cap_pct'] = nasdaq_sym['cap'].apply( lambda x: x/ sum(nasdaq_sym['cap']))
nasdaq_sym_sorted = nasdaq_sym.sort_values(by = "cap", ascending= False)
nasdaq_sym_sorted.head()

,Symbol,Name,lastsale,netchange,pctchange,share_volume,Nasdaq100_points,cap,cap_pct
7,AMZN,Amazon.com Inc.,1837.28,18.42,1.01,3640476,7.6,6.688574e+09,0.125832
11,AAPL,Apple Inc.,197.00,1.31,0.67,18526644,5.2,3.649749e+09,0.068663
88,TSLA,Tesla Inc.,274.96,7.18,2.68,13038257,1.2,3.584999e+09,0.067444
71,NVDA,NVIDIA Corporation,190.95,2.69,1.43,12043649,1.5,2.299735e+09,0.043265
2,AMD,Advanced Micro Devices Inc.,28.98,-0.11,-0.38,65662738,-0.1,1.902906e+09,0.035799


## Get Historical Stock Prices into `json_data`
This might take a while...

In [4]:
#universe = Stock(list(nasdaq_sym_sorted.Symbol)[:100])
blacklist = ['WYNN', 'HAS', 'CTAS']
universe = [sym for sym in list(nasdaq_sym_sorted.Symbol) if not(sym in blacklist)]

start = datetime(2018, 1, 1)
end = datetime(2019, 1, 1)
json_data = get_historical_data( universe, start, end)

## Calculate Sharpe Ratio and put it back into `nasdaq_sym`

In [5]:
def GetSharpe(Symbol, iex_json, outlabel = None):
    '''
    Given a Symbol, Returns
    * return
    * adj sharpe ratio : return/ stdev_returns
    '''
    outDict = {}
    
    if Symbol not in blacklist:
        jd = iex_json[Symbol]
        if any(jd):
            jdf = pd.DataFrame.from_dict(jd, orient='index')

            outDict['return'] = jdf.close[-1]/ jdf.close[0]-1

            jdf['returns'] = jdf['close'] / jdf['close'].shift(1) - 1
            outDict['sigma'] = np.std( jdf['returns'])
            outDict['sharpe'] = outDict['return']/ outDict['sigma']
    
    if any(outDict):
        if outlabel is None:
            return outDict
        else:
            return outDict[outlabel]
    else:
        return np.nan

### Test out the function

In [6]:
GetSharpe('AAPL', json_data,'sharpe')

-3.9038773189955913

In [7]:
nasdaq_sym['sharpe'] = nasdaq_sym['Symbol'].apply(lambda x : GetSharpe(x, json_data,'sharpe'))
nasdaq_sym.head()

,Symbol,Name,lastsale,netchange,pctchange,share_volume,Nasdaq100_points,cap,cap_pct,sharpe
0,ATVI,Activision Blizzard Inc,47.29,0.33,0.70,5976678,0.2,2.826371e+08,0.005317,-11.599855
1,ADBE,Adobe Inc.,267.45,-0.44,-0.16,1970627,-0.2,5.270442e+08,0.009915,12.231808
2,AMD,Advanced Micro Devices Inc.,28.98,-0.11,-0.38,65662738,-0.1,1.902906e+09,0.035799,17.235484
3,ALXN,Alexion Pharmaceuticals Inc.,141.20,1.27,0.91,1208313,0.3,1.706138e+08,0.003210,-8.796487
4,ALGN,Align Technology Inc.,295.88,7.92,2.75,1386161,0.6,4.101373e+08,0.007716,-2.302726


## Visualize

In [8]:
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
# Required for Plot.ly Offline
init_notebook_mode(connected=True)
# Cufflinks bind plotly to pandas dataframe in IPython Notebooks
cf.set_config_file(offline = False, world_readable = True, theme = 'ggplot')


In [9]:
plotly_data = [go.Scatter(
                        y = nasdaq_sym['cap'], 
                        x = nasdaq_sym['sharpe'], 
                        mode = 'markers',
                        text = nasdaq_sym['Symbol']
                    )
              ]
layout = go.Layout(
        title = 'Market Capitalization vs Adjusted Sharpe Ratio',
        yaxis = {'title': 'Market Cap %'},
        xaxis = {'title': 'Adjusted Sharpe Ratio'}
)

fig = go.Figure( data = plotly_data, layout= layout)

iplot(fig)



## Symbol Lookup

In [10]:
myStock = Stock('WDAY')

info = myStock.get_company()
lKeyInfo = ['companyName', 'description', 'exchange']

print('--- Company Description ---')
for key in lKeyInfo:
    print(f'{key}: {info[key]}')

--- Company Description ---
companyName: Workday Inc.
description: Workday Inc is a SaaS provider offering enterprise resource planning, or ERP, software for medium and large enterprises. The company's product portfolio is built around human capital and financial management software.
exchange: Nasdaq Global Select


In [11]:
stats = myStock.get_key_stats()
lKeyStats = [ 'beta', 'dividendRate', 'exDividendDate',
            'priceToBook', 'priceToSales', 'sharesOutstanding',
            'week52high', 'week52low']

StatsDict = {
    'beta': '{:.2f}',
    'dividendRate': '{:.2f}',
    'exDividendDate': '{}',
    'priceToBook': '{:.2f}',
    'priceToSales':'{:.2f}',
    'sharesOutstanding':'{:,.0f}',
    'week52high': '{:.2f}',
    'week52low': '{:.2f}'
}

print('--- Key Stats ---')
for key in StatsDict:
    print( f'{key}: {StatsDict[key].format(stats[key])}')

--- Key Stats ---
beta: 1.79
dividendRate: 0.00
exDividendDate: 0
priceToBook: 21.93
priceToSales: 10.32
sharesOutstanding: 157,000,000
week52high: 200.00
week52low: 117.24
